In [99]:
import pandas as pd
import numpy as np
import ast
import requests

In [5]:
SMILES = pd.read_csv("SMILES.csv")
SMILES.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [77]:
Preds = pd.read_csv("Final_Predictions.csv")
Preds.drop(['Unnamed: 0'], axis = 1, inplace = True)
Preds.rename({'drug': 'Drug'}, axis = 1, inplace = True)

In [78]:
Final= Preds.merge(SMILES, on = "Drug", how = "left")

In [79]:
Final.dropna(how = 'any', inplace = True)

In [80]:
Final['Activity'] = Final['Predictions'].apply(lambda x: 'BCRP_substrate' if x < 0.19 else 'BCRP_inhibitor')

In [81]:
Final.to_csv("Final_Output.csv")

In [82]:
Final.drop(['Predictions'], axis = 1, inplace = True)

In [83]:
Final['Activity'].value_counts()

Activity
BCRP_substrate    4483
BCRP_inhibitor    3611
Name: count, dtype: int64

In [87]:
Final['References'] = Final['References'].apply(lambda x: ast.literal_eval(x))

In [88]:
exploded = Final.explode('References')

In [89]:
unique_values_df = pd.DataFrame(exploded['References'].unique(), columns=['References'])

# Create a dictionary that maps values to the index in the reference DataFrame
value_to_index = {value: idx for idx, value in unique_values_df.reset_index().values}

# Replace the values in the exploded DataFrame with the index from the reference DataFrame
exploded['References_Mapped'] = exploded['References'].map(value_to_index)


In [90]:
exploded

,Drug,References,SMILES,Activity,References_Mapped
2,(+)-catechin,PMC6470784,C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C3)O)O)O,BCRP_inhibitor,0
3,(+)-gallocatechin,PMC6470784,C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C(=C3)O)O)O)O,BCRP_inhibitor,0
4,(+)-naloxone,PMC5725546,C=CCN1CCC23C4C(=O)CCC2(C1CC5=C3C(=C(C=C5)O)O4)O,BCRP_substrate,1
6,(-)-epicatechin,PMC4761526,C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C3)O)O)O,BCRP_inhibitor,2
7,(-)-epigallocatechin gallate,PMC9376584,C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C(=C3)O)O)O...,BCRP_inhibitor,3
...,...,...,...,...,...
18616,zosuquidar,PMC2258302,C1CN(CCN1CC(COC2=CC=CC3=C2C=CC=N3)O)C4C5=CC=CC...,BCRP_inhibitor,4598
18617,zosuquidar hydrochloride,PMC8400349,C1CN(CCN1CC(COC2=CC=CC3=C2C=CC=N3)O)C4C5=CC=CC...,BCRP_inhibitor,5815
18618,zosuquidar trihydrochloride,PMC5713259,C1CN(CCN1CC(COC2=CC=CC3=C2C=CC=N3)O)C4C5=CC=CC...,BCRP_substrate,3034
18619,zoxazolamine,PMC4963026,C1=CC2=C(C=C1Cl)N=C(O2)N,BCRP_substrate,1001


In [91]:
mappedpmcid =exploded.groupby('Drug').agg({
    'References_Mapped': lambda x: list(x.unique())
}).reset_index()

In [92]:
Final = Final.merge(mappedpmcid, how = "inner", on= "Drug")

In [136]:
cid_1 = pd.read_csv("SMILES_CID_1.csv")
cid_1.drop(['Unnamed: 0'], axis = 1, inplace = True)
cid_1 = cid_1.drop(['Synonyms'], axis = 1).dropna(how= "any")
cid_1['CID'] = cid_1['SMILES'].apply(lambda x: x.split(",")[1])
cid_1.rename({"drug": 'Drug'}, axis = 1, inplace = True)

In [142]:
cid_1

,Drug,SMILES,CID
0,tyrosine,"C1=CC(=CC=C1CC(C(=O)O)N)O,6057",6057
1,Chloroquine,"CCN(CC)CCCC(C)NC1=C2C=CC(=CC2=NC=C1)Cl,2719",2719
2,Ciclosporin,CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(...,5284373
5,ATP,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)COP(=O)(O)OP(...,5957
6,cisplatin,"N.N.Cl[Pt]Cl,5702198",5702198
...,...,...,...
9377,17-allylamino-17-demethoxygeldanamycin,CC1CC(C(C(C=C(C(C(C=CC=C(C(=O)NC2=CC(=O)C(=C(C...,6440175
9378,tanespimycin,CC1CC(C(C(C=C(C(C(C=CC=C(C(=O)NC2=CC(=O)C(=C(C...,6505803
9379,Geldanamycin,CC1CC(C(C(C=C(C(C(C=CC=C(C(=O)NC2=CC(=O)C(=C(C...,5288382
9380,Tanespimycin,CC1CC(C(C(C=C(C(C(C=CC=C(C(=O)NC2=CC(=O)C(=C(C...,6505803


In [137]:
cid_2 = pd.read_csv("SMILES_CID_2.csv")
cid_2.drop(['Unnamed: 0'], axis = 1, inplace = True)
cid_2 = cid_2.drop(['Synonyms'], axis = 1).dropna(how= "any")
cid_2['CID'] = cid_2['SMILES'].apply(lambda x: x.split(",")[1])
cid_2.rename({"drug": 'Drug', 'CID': 'CID_2'}, axis = 1, inplace = True)

In [146]:
Final = Final.merge(cid_1, how = "left", on = 'Drug')
Final = Final.merge(cid_2, how = "left", on= 'Drug')

MergeError: Passing 'suffixes' which cause duplicate columns {'SMILES_x'} is not allowed.

In [148]:
Final.drop(['SMILES_y', 'SMILES'], axis = 1, inplace = True)

In [149]:
Final

,Drug,References,SMILES_x,Activity,References_Mapped,CID,CID_2
0,(+)-catechin,[PMC6470784],C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C3)O)O)O,BCRP_inhibitor,[0],NaN,9064
1,(+)-gallocatechin,[PMC6470784],C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C(=C3)O)O)O)O,BCRP_inhibitor,[0],NaN,65084
2,(+)-naloxone,[PMC5725546],C=CCN1CCC23C4C(=O)CCC2(C1CC5=C3C(=C(C=C5)O)O4)O,BCRP_substrate,[1],NaN,5491858
3,(-)-epicatechin,[PMC4761526],C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C3)O)O)O,BCRP_inhibitor,[2],NaN,72276
4,(-)-epigallocatechin gallate,[PMC9376584],C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C(=C3)O)O)O...,BCRP_inhibitor,[3],65064,NaN
...,...,...,...,...,...,...,...
8089,zosuquidar,"[PMC10575401, PMC10457872, PMC10353158, PMC103...",C1CN(CCN1CC(COC2=CC=CC3=C2C=CC=N3)O)C4C5=CC=CC...,BCRP_inhibitor,"[4088, 8501, 1520, 1561, 5197, 1588, 1595, 159...",3036703,NaN
8090,zosuquidar hydrochloride,[PMC8400349],C1CN(CCN1CC(COC2=CC=CC3=C2C=CC=N3)O)C4C5=CC=CC...,BCRP_inhibitor,[5815],134159522,NaN
8091,zosuquidar trihydrochloride,[PMC5713259],C1CN(CCN1CC(COC2=CC=CC3=C2C=CC=N3)O)C4C5=CC=CC...,BCRP_substrate,[3034],NaN,153997
8092,zoxazolamine,[PMC4963026],C1=CC2=C(C=C1Cl)N=C(O2)N,BCRP_substrate,[1001],NaN,6103


In [151]:
Final['PubChem_CID'] = Final['CID'].combine_first(Final['CID_2'])

In [153]:
Final.drop(['CID', 'CID_2'], inplace = True, axis = 1)

In [155]:
Final.rename({"SMILES_x": 'SMILES'}, axis = 1, inplace = True)

In [157]:
Final.to_csv("training.tsv", "\t")

In [115]:
unique_values_df.to_csv("references.tsv", '\t')

In [109]:
Final['Activity'].value_counts()['BCRP_inhibitor']*100/len(Final) 

44.61329379787497

In [111]:
metrics_summary = pd.DataFrame({'Team_Name': "Mihir Padsumbiya", 
                               'Team_Contact': 'mxp210085@utdallas.edu', 
                               'Number of Samples': len(Final),
                               'Number of References': len(unique_values_df),
                               'Number of Inhibitors': Final['Activity'].value_counts()['BCRP_inhibitor'],
                               'Number of Substrate': Final['Activity'].value_counts()['BCRP_substrate'],
                               'Percent_minority': 44.6,
                               'Number of Smiles': len(Final),
                               'Percent_Smiles': 1.0}, index = [1])

In [114]:
metrics_summary.to_csv("metrics_summary.tsv", "\t")